# Running agent on pretrained weights

In [1]:
from unityagents import UnityEnvironment
from collections import deque
from ddpg_agent import Agent
import torch
import numpy as np

Load environment

In [2]:
env = UnityEnvironment(file_name="./Reacher_Windows_x86_64/Reacher.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

Initialize agent and necessary variables

In [4]:
params = {
"BUFFER_SIZE":int(1e5),  # replay buffer size
"BATCH_SIZE":128,       # minibatch size
"GAMMA":0.99,            # discount factor
"TAU":1e-3,              # for soft update of target parameters
"LR_ACTOR":0.0001,         # learning rate of the actor 
"LR_CRITIC":0.0001,        # learning rate of the critic
"WEIGHT_DECAY":0.0001   # L2 weight decay
}

agent = Agent(n = 20, state_size=33, action_size=4, random_seed=42, params=params)
# Load the weights from file
agent.actor_local.load_state_dict(torch.load('actor.pth'))
agent.critic_local.load_state_dict(torch.load('critic.pth'))
agent.actor_target.load_state_dict(torch.load('actor.pth'))
agent.critic_target.load_state_dict(torch.load('critic.pth'))
score = [] 
env_info = env.reset(train_mode=False)[brain_name]
states = env_info.vector_observations

Let the agent do it's job

In [5]:
while True:
    actions = agent.act(states)
    env_info = env.step(actions)[brain_name]            # send all actions to tne environment
    next_states = env_info.vector_observations          # get next states
    rewards = env_info.rewards                       # get reward
    dones = env_info.local_done                         # see if episode finished
    agent.step(states, actions, rewards, next_states, dones)
    states = next_states
    score += rewards                           # update the score 
    if np.any(dones):
        break 

print("Score: {}".format(np.mean(score)))

Score: 0.03694105811536193


In [6]:
env.close()